# Control de versiones con git

**Curso**: Ciencia de Datos en Python

**Catedrático**: Ing. Luis Leal

**Estudiante**: Dany Rafael Díaz Lux (21000864)

**Enunciado**: Tomar la [lección del MIT de git](https://missing.csail.mit.edu/2020/version-control/) y en un jupyter notebook entregar un ensayo acerca de git, es decir: un resumen de lo que ha aprendido, conclusiones, comentarios y ejemplos propios, esto usando markdown en el notebook , para ejemplificar cualquier comando de git pueden usar [magic cells](https://jakevdp.github.io/PythonDataScienceHandbook/01.03-magic-commands.html), un tipo especial de celdas de jupyter que nos permite escribir comandos que no son de python si no de nuestro sistema.

**Fecha de entrega**: miércoles 10 de febrero.

## Resumen

### Introducción
Los sistemas de control de versiones (VCSs por sus siglas en inglés) ayudan a llevar seguimiento de los cambios que se aplican a un conjunto de archivos. Ayudan mucho en proyectos donde colaboran muchas personas en diferentes archivos, o incluso en cuando estás trabajando en solitario porque VCSs ayudan a contestar preguntas importantes como:
* ¿__Quién__ cambió este archivo?
* ¿__Cuándo__ fue introducido este cambio?
* ¿__Por qué__ fue cambiada esta línea?
* _Muchas más_...

### Entendiendo git
git es un VCS muy popular que regularmente se empieza a usar sin entender por completo cómo funciona, esto puede conllevar a utilizar sólo los comandos más básicos de git sin mayor comprensión de lo que pasa con nuestros archivos y nos impiden resolver problemas más complejos que pueden presentarse más adelante.

Para mejorar nuestra compresión de git iniciaremos con una explicación de su modelo de datos.

### Modelo de datos de git

#### _Snapshots_ (instantáneas)
git modela la historia de un conjunto de archivos, dentro de un folder principal, como **una serie de _snapshots_**. Para entender un _snapshot_, se deben comprender los siguientes términos:
* _blob_: un archivo, un conjunto de bytes.
* _tree_ (árbol): un folder, que puede contener _blobs_ o _trees_.

Un _snapshot_ es el folder principal (que contiene todos los _blobs_ y _trees_) que está siendo registrado por git.

### Modelando la historia = relacionando _snapshots_
Para manejar la historia de todos los archivos, git usa un _Grafo dirigido acíclico_ (DAG por sus siglas en inglés) de _snapshots_, que básicamente significa que cada _snapshot_ en git hará referencia a uno o varios _snapshots_ padres. Como lo muestra la siguiente imagen:

![Grafo dirigido acíclico de git](https://www.oreilly.com/library/view/git-pocket-guide/9781449327507/images/gtpg_0101.png "Grafo dirigido acíclico de git")

_Imagen obtenida del de la página web: [Chapter 1. Understanding Git](https://www.oreilly.com/library/view/git-pocket-guide/9781449327507/ch01.html), como se puede observar, cada nodo en el grafo (que representa un snapshot) apunta al menos a un padre, con la excepción del nodo inicial que representa el inicio de la historia de los archivos en git._

Cada _snapshot_ se realiza por medio de un _commit_ (comprometer) en git, que guarda los contenidos de todos los archivos en ese momento específico.

### Objetos y direccionamiento de contenido
Un objeto puede ser un _blob_ (archivo), _tree_ (folder), o un _commit_ (instantánea). En el almacén de datos de git todo el contenido de los  objetos pueden ser obtenidos por un identificador generado por la aplicación del algoritmo [SHA1](https://en.wikipedia.org/wiki/SHA-1) en dicho objeto.

Es importante señalar que cuando un objeto hace referencia a otro objeto, **no posee el contenido del objeto referenciado, sólo posee el identificador a ese objeto**. Así por ejemplo, un _tree_ (folder) sólo posee los identificadores a los objetos que contiene.

### Referencias
Los identificadores generados por el algoritmo **SHA1** están formados por 40 caracteres alfanuméricos (como `a43913adf3occmce...`), lo cual no es conveniente para las personas. Por lo cual git permite utilizar **referencias** que son nombres que podemos darle a cualquier _commit_, como por ejemplo `Version2021`. Estas referencias pueden ser actualizadas para apuntar a nuevos o anteriores _commits_.

Existe una referencia especial **que indica donde nos encontramos siempre** en nuestro grafo de git, esta referencia se llama **HEAD**.

### Repositorios
Un repositorio de git son los objetos de datos y referencias. En eso consiste básicamente el modelo de datos de git. Todos los comandos de git hacen manipulaciones al _DAG_ por medio de agregar objetos y agregar o actualizar referencias.

### _Staging area_ (área de preparación)
_Staging area_ es aquella en la cual se incluyen todos los cambios que deseamos incluir en un _commit_. El objetivo de _staging area_ es realizar commit con el estado de los archivos que deseamos. Por ejemplo, si tenemos 2 archivos modificados, pero sólo deseamos incluir las modificaciones de un archivo en el próximo _commit_, incluimos sólo este archivo en nuestra _staging area_. El otro archivo no incluido se puede dejar para más adelante o descartar de acuerdo a nuestras necesidades.

## Ejemplos
A continuación se mostrarán diversos comandos de git para indicar su funcionamiento y utilidad.

**_Nota: 
Todos los comandos a continuación fueron ejecutados en un sistema Windows 10, con diversas utilidades adicionales instaladas (como git bash), por lo que si un comando no funciona adecuadamente podría deberse a problemas específicos de sistema operativo o utilidades que no se tienen instaladas. Además los comandos se deben ejecutar en el orden que se muestran a continuación, pues algunos de ellos dependen de comandos ejecutados anteriormente._**

### git init
Crea un nuevo repositorio de git

In [7]:
%sx git init _nuevo_repositorio_test_

['Initialized empty Git repository in C:/Users/Dany/Documents/01 Dany/MaestrÃ\xada/Python para ciencia de datos/python-ciencia-de-datos/Tarea 2 - git y control de versiones/_nuevo_repositorio_test_/.git/']

### git status
Señala el estado de nuestro repositorio actualmente. Qué archivos han cambiado. Qué archivos están dentro y fuera de _staging area_. etc.

In [18]:
# Enter to the new repository created
%cd _nuevo_repositorio_test_

# Create a new file in the repository initialized
%sx touch archivo_de_prueba.txt

# Add some content to the file
%sx echo Listado de cosas: > archivo_de_prueba.txt
%sx echo Sol >> archivo_de_prueba.txt

# Ejecutar git status para ver el estado actual del repositorio
%sx git status

C:\Users\Dany\Documents\01 Dany\Maestría\Python para ciencia de datos\python-ciencia-de-datos\Tarea 2 - git y control de versiones\_nuevo_repositorio_test_


['On branch master',
 '',
 'No commits yet',
 '',
 'Untracked files:',
 '  (use "git add <file>..." to include in what will be committed)',
 '',
 '\tarchivo_de_prueba.txt',
 '',
 'nothing added to commit but untracked files present (use "git add" to track)']

## Comentarios personales

## Conclusiones